In [1]:
import os 
import numpy as np
import pandas as pd
import ast
# Generate the training data
import cv2
import gc

In [2]:
out1 = pd.read_csv('best single model vggbnw_fcn_en.csv', index_col=0)
out2 = pd.read_csv('even better public score until now.csv', index_col=0)
out3 = pd.read_csv('gbm.csv', index_col=0)
out4 = pd.read_csv('inceptionV3.csv', index_col=0)
out5 = pd.read_csv('statiol csv pytorch senet ensemble.csv', index_col=0)
out6 = pd.read_csv('vgg16.csv', index_col=0)
out7 = pd.read_csv('vgg16-BatchNorm-FC.csv', index_col=0)
out8 = pd.read_csv('vgg19.csv', index_col=0)
out9 = pd.read_csv('xception.csv', index_col=0)
out10 = pd.read_csv('resnet50.csv', index_col=0)
out11 = pd.read_csv('my best single model- simple cnn.csv', index_col=0)
out12 = pd.read_csv('stacking_from_staked.csv', index_col=0)
out13 = pd.read_csv('inceptionV3-test-data.csv', index_col=0)
out14 = pd.read_csv('vgg16_test.csv', index_col=0)
out15 = pd.read_csv('vgg16-BatchNorm-test-data.csv', index_col=0)
out16 = pd.read_csv('vgg19-test-data.csv', index_col=0)
out17 = pd.read_csv('xception-test-data.csv', index_col=0)
out18 = pd.read_csv('ResNet50-test-data.csv', index_col=0)
out19 = pd.read_csv('ensemble-all-test-nontest.csv', index_col=0)

out20 = pd.read_csv('InceptionV3-test-data-final.csv', index_col=0)
out21 = pd.read_csv('vgg16_with_test_data-final.csv', index_col=0)
out22 = pd.read_csv('vgg16-batchnorm-test-data-final.csv', index_col=0)
out23 = pd.read_csv('vgg19-test-data-final.csv', index_col=0)
out24 = pd.read_csv('xception-test-data-final.csv', index_col=0)

concat_sub = pd.concat([out1, out2, out3, out4, out5, out6, out7, out8, out9, out10,
                       out11, out12, out13, out14, out15, out16, out17, out18, out19,
                       out20, out21, out22, out23, out24], axis=1)
cols = list(map(lambda x: "is_iceberg_" + str(x), range(len(concat_sub.columns))))
concat_sub.columns = cols
concat_sub.reset_index(inplace=True)
concat_sub['id'] = concat_sub['index']

In [3]:
# get the data fields ready for stacking
concat_sub['is_iceberg_max'] = concat_sub.iloc[:, 1:25].max(axis=1)
concat_sub['is_iceberg_min'] = concat_sub.iloc[:, 1:25].min(axis=1)
concat_sub['is_iceberg_mean'] = concat_sub.iloc[:, 1:25].mean(axis=1)
concat_sub['is_iceberg_median'] = concat_sub.iloc[:, 1:25].median(axis=1)

In [4]:
# set up cutoff threshold for lower and upper bounds, easy to twist 
cutoff_lo = 0.8
cutoff_hi = 0.2
concat_sub.corr()

,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3,is_iceberg_4,is_iceberg_5,is_iceberg_6,is_iceberg_7,is_iceberg_8,is_iceberg_9,...,is_iceberg_18,is_iceberg_19,is_iceberg_20,is_iceberg_21,is_iceberg_22,is_iceberg_23,is_iceberg_max,is_iceberg_min,is_iceberg_mean,is_iceberg_median
is_iceberg_0,1.000000,0.854066,0.563222,0.874871,0.884432,0.902464,0.826055,0.898955,0.821536,0.876818,...,0.960847,0.805762,0.827528,0.855890,0.838291,0.812419,0.625456,0.785234,0.912328,0.883463
is_iceberg_1,0.854066,1.000000,0.430573,0.782052,0.952768,0.901841,0.933050,0.919714,0.945083,0.872653,...,0.963275,0.940048,0.955592,0.954635,0.956223,0.940826,0.489172,0.906826,0.971138,0.979512
is_iceberg_2,0.563222,0.430573,1.000000,0.661887,0.555389,0.533538,0.393239,0.534700,0.379435,0.564442,...,0.522123,0.340202,0.361074,0.391232,0.369895,0.348904,0.938072,0.367249,0.514525,0.433115
is_iceberg_3,0.874871,0.782052,0.661887,1.000000,0.844864,0.832906,0.768524,0.831236,0.760508,0.860606,...,0.860853,0.754595,0.743862,0.780075,0.750332,0.749410,0.707116,0.740827,0.857736,0.809574
is_iceberg_4,0.884432,0.952768,0.555389,0.844864,1.000000,0.945271,0.912097,0.951350,0.915361,0.915997,...,0.954998,0.888810,0.914623,0.925841,0.918320,0.892480,0.617085,0.871634,0.970380,0.956379
is_iceberg_5,0.902464,0.901841,0.533538,0.832906,0.945271,1.000000,0.877347,0.973178,0.877475,0.904175,...,0.937494,0.826572,0.880067,0.884638,0.891536,0.833729,0.596369,0.814131,0.941891,0.928273
is_iceberg_6,0.826055,0.933050,0.393239,0.768524,0.912097,0.877347,1.000000,0.901133,0.981699,0.851241,...,0.914068,0.933896,0.944010,0.944873,0.941233,0.938198,0.462198,0.918261,0.954893,0.962355
is_iceberg_7,0.898955,0.919714,0.534700,0.831236,0.951350,0.973178,0.901133,1.000000,0.903137,0.906792,...,0.945149,0.859673,0.899810,0.908417,0.914030,0.866445,0.600740,0.844787,0.958921,0.945638
is_iceberg_8,0.821536,0.945083,0.379435,0.760508,0.915361,0.877475,0.981699,0.903137,1.000000,0.850516,...,0.918054,0.946796,0.956471,0.956273,0.954612,0.950046,0.455838,0.921413,0.960247,0.969553
is_iceberg_9,0.876818,0.872653,0.564442,0.860606,0.915997,0.904175,0.851241,0.906792,0.850516,1.000000,...,0.909500,0.818972,0.843680,0.872471,0.853021,0.831117,0.625073,0.818064,0.921929,0.895999


In [5]:
sub_base = pd.read_csv('stacking_from_staked.csv')

In [6]:
concat_sub['is_iceberg_base'] = sub_base['is_iceberg']
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:25] > cutoff_lo, axis=1), 
                                    concat_sub['is_iceberg_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:25] < cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'], 
                                             concat_sub['is_iceberg_base']))
concat_sub[['id', 'is_iceberg']].to_csv('ensemble-all-test-nontest-final.csv', 
                                        index=False, float_format='%.6f')

In [32]:
solution1 = pd.read_csv('ensemble-all-test-nontest.csv',index_col=0)
solution2 = pd.read_csv('stacking_from_staked.csv', index_col=0)
solution3 = pd.read_csv('even better public score until now.csv', index_col=0)
solution4 = pd.read_csv('best single model vggbnw_fcn_en.csv', index_col=0)


In [33]:
solution1['is_iceberg2'] = solution2['is_iceberg']
solution1['is_iceberg3'] = solution3['is_iceberg']
solution1['is_iceberg4'] = solution4['is_iceberg']
solution1.corr()

,is_iceberg,is_iceberg2,is_iceberg3,is_iceberg4
is_iceberg,1.000000,0.999213,0.963275,0.960847
is_iceberg2,0.999213,1.000000,0.963411,0.962236
is_iceberg3,0.963275,0.963411,1.000000,0.854066
is_iceberg4,0.960847,0.962236,0.854066,1.000000


In [34]:
concat_sub_thresh = solution1.where(
    ((solution1['is_iceberg'] > 0.97) &
    (solution1['is_iceberg2'] > 0.97) &
    (solution1['is_iceberg3'] > 0.97) &
    (solution1['is_iceberg4'] > 0.97)) |
    ((solution1['is_iceberg'] < 0.03) &
    (solution1['is_iceberg2'] < 0.03) &
    (solution1['is_iceberg3'] < 0.03) &
    (solution1['is_iceberg4'] < 0.03)), axis=1) 

concat_sub_thresh.head()

,is_iceberg,is_iceberg2,is_iceberg3,is_iceberg4
id,,,,
000c65d7,NaN,NaN,NaN,NaN
001680af,NaN,NaN,NaN,NaN
00193999,NaN,NaN,NaN,NaN
00247e48,0.999995,0.996103,0.999,0.993207
00503faa,0.000000,0.001401,0.001,0.001801


In [35]:
concat_sub_thresh = concat_sub_thresh.dropna(axis=0, how='any')
concat_sub_thresh['is_iceberg1'] = concat_sub_thresh['is_iceberg']

In [39]:
for i, num in enumerate(is_iceberg1):
    if num > .5:
        is_iceberg1[i] = 1
    else:
        is_iceberg1[i] = 0

In [40]:
concat_array = np.concatenate([id_[:,np.newaxis],
                        is_iceberg1[:,np.newaxis]],axis=-1)

df = pd.DataFrame(concat_array, columns=['id', 'is_iceberg'])

test = pd.read_json('data/processed/test.json')

df_test_labels = pd.merge(test, df, on='id')


In [41]:
df = pd.DataFrame(concat_array, columns=['id', 'is_iceberg'])
df.head(500)

,id,is_iceberg
0,00247e48,1
1,00503faa,0
2,00542824,0
3,005d1a8d,1
4,006369ec,0
5,006655ad,1
6,006abe61,0
7,008ae7b6,1
8,009b8524,0
9,00a92ec1,0


In [43]:
test = pd.read_json('data/processed/test.json')

In [52]:
train_with_testdata_csv = pd.read_csv('df_train_with_new_test.csv')

In [54]:
target_train = df_train_with_new_test['is_iceberg']
test = pd.read_json('data/processed/test.json')

test['inc_angle'] = pd.to_numeric(test['inc_angle'], errors='coerce')
x_angle = df_train_with_new_test['inc_angle']
test['inc_angle'] = pd.to_numeric(test['inc_angle'], errors='coerce')
x_test_angle = test['inc_angle']

In [59]:
x_band_1 = np.array([cv2.resize(np.array(band).astype(np.float32).reshape(75, 75), (3*75, 3*75), interpolation = cv2.INTER_CUBIC)
                                for band in df_train_with_new_test['band_1']])
x_band_2 = np.array([cv2.resize(np.array(band).astype(np.float32).reshape(75, 75), (3*75, 3*75), interpolation = cv2.INTER_CUBIC)
                                for band in df_train_with_new_test['band_2']])
x_band_3 = (x_band_1 + x_band_2)/2
# x_band_3 = np.array([np.full((75, 75), angle).astype(np.float32) for angle in train['inc_angle]])
x_train2 = np.concatenate([x_band_1[:,:,:,np.newaxis],
                         x_band_2[:,:,:,np.newaxis],
                         x_band_3[:,:,:,np.newaxis]], axis=-1)

In [62]:
x_band_test_1 = np.array([cv2.resize(np.array(band).astype(np.float32).reshape(75, 75), (3*75, 3*75), interpolation = cv2.INTER_CUBIC)
                                for band in test['band_1']])
x_band_test_2 = np.array([cv2.resize(np.array(band).astype(np.float32).reshape(75, 75), (3*75, 3*75), interpolation = cv2.INTER_CUBIC)
                                for band in test['band_2']])
x_band_test_3 = (x_band_test_1 + x_band_test_2)/2
# x_band_3 = np.array([np.full(75, 75), angle).astype(np.float32) for angle in test['inc_angle]])
x_test = np.concatenate([x_band_test_1[:,:,:,np.newaxis],
                        x_band_test_2[:,:,:,np.newaxis],
                        x_band_test_3[:,:,:,np.newaxis]],axis=-1)

In [39]:
np.save('x_test.npy', x_test)
x_test = np.load("x_test_225_225_resize.npy")
x_test.shape

(8424, 225, 225, 3)

In [40]:
np.save('x_train.npy', x_train2)
x_test = np.load("x_test_225_225_resize.npy")
x_test.shape

(8424, 225, 225, 3)

In [41]:
x_train = np.load("x_train.npy")
x_test = np.load("x_test.npy")
print(x_test.shape, x_train.shape)

(8424, 75, 75, 3) (4929, 75, 75, 3)
